# Welcome to Week 2!

## Frontier Model APIs

In Week 1, we used multiple Frontier LLMs through their Chat UI, and we connected with the OpenAI's API.

Today we'll connect with the APIs for Anthropic and Google, as well as OpenAI.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Important Note - Please read me</h2>
            <span style="color:#900;">I'm continually improving these labs, adding more examples and exercises.
            At the start of each week, it's worth checking you have the latest code.<br/>
            First do a <a href="https://chatgpt.com/share/6734e705-3270-8012-a074-421661af6ba9">git pull and merge your changes as needed</a>. Any problems? Try asking ChatGPT to clarify how to merge - or contact me!<br/><br/>
            After you've pulled the code, from the llm_engineering directory, in an Anaconda prompt (PC) or Terminal (Mac), run:<br/>
            <code>conda env update --f environment.yml --prune</code><br/>
            Or if you used virtualenv rather than Anaconda, then run this from your activated environment in a Powershell (PC) or Terminal (Mac):<br/>
            <code>pip install -r requirements.txt</code>
            <br/>Then restart the kernel (Kernel menu >> Restart Kernel and Clear Outputs Of All Cells) to pick up the changes.
            </span>
        </td>
    </tr>
</table>
<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Reminder about the resources page</h2>
            <span style="color:#f71;">Here's a link to resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

## Setting up your keys

We will use the models through cloud providers, you will need to have credentials for AWS and Azure for this.

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
AZURE_OPENAI_API_KEY=xxxx
AZURE_OPENAI_ENDPOINT=https://example.openai.azure.com
AWS_ACCESS_KEY_ID=xxxx
AWS_SECRET_ACCESS_KEY=xxxx
AWS_SESSION_TOKEN=xxxx
AWS_REGION=us-west-2
OPENAI_BASE_URL=https://localhost:11434/v1
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
import json
import boto3
from IPython.display import Markdown, display, update_display

In [ ]:
# import for google
# in rare cases, this seems to give an error on some systems. Please reach out to me if this happens,
# or you can feel free to skip Gemini - it's the lowest priority of the frontier models that we use

import google.generativeai

In [ ]:
# load the environment variables
load_dotenv()

In [ ]:
# Test that AZURE works
AZURE_MODEL = "gpt-4o"
client_azure = AzureOpenAI(
     api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version="2024-08-01-preview",
)
messages = [
    {
        "role": "user",
        "content": "ping"
    }
]
response = client_azure.chat.completions.create(model=AZURE_MODEL, messages=messages)
print(response.choices[0].message.content)

In [ ]:
# Test that AWS works
AWS_MODEL = "anthropic.claude-3-sonnet-20240229-v1:0"
session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime', region_name='us-east-1')
# AWS Messages are a bit more complex
aws_message = {
    "role": "user",
    "content": [
        { "text": "how are you doing" } 
    ],
}
response = bedrock.converse(
    modelId=AWS_MODEL,
    inferenceConfig={
    "maxTokens": 2000,
    "temperature": 0
    },
    messages=[aws_message],
)
print(response['output']['message']['content'][0]['text'])

In [ ]:
# Test ollama using OpenAI API
OLLAMA_MODEL='qwen2.5'
print(os.getenv('OPENAI_BASE_URL'))
client_ollama = OpenAI(
        base_url=os.getenv('OPENAI_BASE_URL'),
        api_key='123'
    )
response = client_ollama.chat.completions.create(model=OLLAMA_MODEL, messages=messages)
print(response.choices[0].message.content)

In [ ]:
# Connect to OpenAI, Anthropic and Google
# All 3 APIs are similar
# Having problems with API files? You can use openai = OpenAI(api_key="your-key-here") and same for claude
# Having problems with Google Gemini setup? Then just skip Gemini; you'll get all the experience you need from GPT and Claude.

google.generativeai.configure()

## Asking LLMs to tell a joke

It turns out that LLMs don't do a great job of telling jokes! Let's compare a few models.
Later we will be putting LLMs to better use!

### What information is included in the API

Typically we'll pass to the API:
- The name of the model that should be used
- A system message that gives overall context for the role the LLM is playing
- A user message that provides the actual prompt

There are other parameters that can be used, including **temperature** which is typically between 0 and 1; higher for more random output; lower for more focused and deterministic.

In [ ]:
system_message = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# GPT-4o
def call_azure(model=AZURE_MODEL, temp=0.5):
    openai = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-08-01-preview",
    )
    completion = openai.chat.completions.create(model=model, messages=prompts, temperature=temp)
    return completion.choices[0].message.content
print(call_azure('gpt-4o'))

In [ ]:
# GPT-4o-mini
# Temperature setting controls creativity

print(call_azure('gpt-4o-mini', temp=0.7))

In [ ]:
# GPT-4o

print(call_azure('gpt-4o', temp=0.4))

In [ ]:
# AWS with Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

def call_aws(model=AWS_MODEL, temp=0.5):
    aws_message = {
    "role": "user",
    "content": [
        { "text": user_prompt } 
        ],
    }
    sys_message = [ { "text": system_message } ]
    session = boto3.Session()
    bedrock = session.client(service_name='bedrock-runtime', region_name='us-east-1')
    response = bedrock.converse(
        modelId=model,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": temp
        },
        messages=[aws_message],
        system=sys_message
    )
    return response['output']['message']['content'][0]['text']
print(call_aws(AWS_MODEL))

In [ ]:
# AWS with Claude 3.5 Sonnet
# Now let's add in streaming back results
def call_aws_stream(model=AWS_MODEL, temp=0.5):
    aws_message = {
    "role": "user",
    "content": [
        { "text": user_prompt } 
        ],
    }
    sys_message = [ { "text": system_message } ]
    response = bedrock.converse_stream(
        modelId=model,
        inferenceConfig={
        "maxTokens": 2000,
        "temperature": temp
    },
        system=sys_message,
        messages=[aws_message],
    )
    stream = response.get('stream')
    reply = ""
    for event in stream:
        if "contentBlockDelta" in event:
            text = event["contentBlockDelta"]["delta"]['text']
            print(text, end="", flush=True)
call_aws_stream(AWS_MODEL, temp=0.7)

In [ ]:
# Call Ollama
def call_ollama_stream(model=OLLAMA_MODEL, temp=0.5):
    openai = OpenAI(
        base_url=os.getenv('OPENAI_BASE_URL'),
        api_key='123'
    )
    stream = openai.chat.completions.create(model=model, messages=prompts, temperature=temp, stream=True)
    for chunk in stream:
        if chunk.choices:
            text = chunk.choices[0].delta.content or ''
            print(text, end="", flush=True)
call_ollama_stream(OLLAMA_MODEL)

In [ ]:
# The API for Gemini has a slightly different structure

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [ ]:
# Have it stream back results in markdown

def call_azure_stream(model=AZURE_MODEL, temp=0.5):
    openai = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-08-01-preview",
    )
    return openai.chat.completions.create(model=model, messages=prompts, temperature=temp, stream=True)
stream = call_azure_stream('gpt-4o-mini', temp=0.7)
reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    if chunk.choices:
        reply += chunk.choices[0].delta.content or ''
        reply = reply.replace("```","").replace("markdown","")
        update_display(Markdown(reply), display_id=display_handle.display_id)

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [ ]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "anthropic.claude-3-sonnet-20240229-v1:0"

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [ ]:
def call_gpt():
    azure_client = AzureOpenAI(
        api_key=os.getenv('AZURE_OPENAI_API_KEY'),
        azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
        api_version="2024-08-01-preview",
    )
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = azure_client.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    session = boto3.Session()
    bedrock = session.client(service_name='bedrock-runtime', region_name='us-east-1')
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": [{"text": gpt }]})
        messages.append({"role": "assistant", "content": [{"text": claude_message }]})
    messages.append({"role": "user", "content": [{"text": gpt_messages[-1] }]})
    response = bedrock.converse(
        modelId=claude_model,
        system=[{"text":claude_system}],
        messages=messages,
        inferenceConfig={
            "maxTokens": 2000,
            "temperature": 0
        },
    )
    return response['output']['message']['content'][0]['text']

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions.

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>